In [342]:
import time
R.<x,y,z> = PolynomialRing(QQ, order = 'lex')

In [2]:
# Syzygy polynomial computation as in Cox, Little, and O'Shea

def S_poly(f,g):
    alpha = f.lt()
    beta = g.lt()
    gamma = lcm(alpha,beta)
    return R((gamma / alpha)*f - (gamma / beta)*g) # fails to have lt() without R(.)

In [3]:
# Multivariate division algorithm

def mdiv(f,divisors):
    s = len(divisors)
    q = [0 for i in range(s)]
    r = 0
    p = f
    while p != 0:
        i = 0
        div_occ = False
        while (i < s) and (div_occ == False):
            ltdi = divisors[i].lt()
            ltp = p.lt()
            if ltdi.divides(ltp):
                update = ltp / ltdi
                q[i] += update
                p -= R(update*divisors[i]) # without R(.), p is in fraction field, where lt() command fails
                div_occ = True
            else:
                i += 1
        if div_occ == False:
            r = r + p.lt()
            p = p - p.lt()
    return (q,r)


# Compute the remainder of a multivariate polynomial "f" after division by a list of polynomials "divisors"

def get_rem(f,divisors):
    return mdiv(f,divisors)[1]

In [640]:
# find l where l is the row in the Gröbner basis at the beginning of Theorem 5

def get_l_level(f):
    degs = dict(zip([i+1 for i in range(len(R.gens()))], f.degrees()))
    l = 0
    for (key,val) in degs.items():
        if val != 0:
            l = key
            break
    return l  

# Implementing Corollary 1 in Caminata and Gorla

def system_solve(f): # takes a list f = [f_1, ..., f_s] of multivariate polynomials as input
    ring_vars = dict(zip([i+1 for i in range(len(R.gens()))], R.gens()))
    I = Ideal(f)
    G = I.groebner_basis()
    partial_sols = [[]]
    completed_ls = []
    for g in G[::-1]:
        l = get_l_level(g)
        if l not in completed_ls:
            completed_ls.append(l)
            working_polys = []
            for h in G:
                if get_l_level(h) == l:
                    working_polys.append(h)
            new_partials = []
            for partial in partial_sols:
                specializations = []
                for poly in working_polys:
                    degs = dict(zip([i+1 for i in range(len(R.gens()))], poly.degrees()))
                    l = 0
                    for (key,val) in degs.items():
                        if val != 0:
                            l = key
                            break
                    substitutions = dict(zip(R.gens()[l:], partial))
                    specializations.append(poly.subs(substitutions))
                p = gcd(specializations)
                print(p)
                roots = p.univariate_polynomial().roots()
                for root in roots:
                    new_sol = [root[0]] + partial
                    new_partials.append(new_sol)
            partial_sols = new_partials
    return([tuple(sol) for sol in partial_sols])

# Example from Minko's Thesis
R.<x,y,z,h> = PolynomialRing(GF(7), order = 'lex')
f1 = x^5 + y^5 + z^5 - 1
f2 = x^3 + y^3 + z^2 - 1
f3 = y^6 - 1
f4 = z^6 - 1
f_list = [f1*f1*f1, f1*f1*f2, f1*f1*f3, f1*f1*f4, f1*f2*f2, f1*f2*f3, f1*f2*f4, f1*f3*f3, f1*f3*f4, f1*f4*f4, f2*f2*f2, f2*f2*f3, f2*f2*f4, f2*f3*f3, f2*f3*f4, f2*f4*f4, f3*f3*f3, f3*f3*f4, f3*f4*f4, f4*f4*f4, x^7 - x, y^7 - y, z^7 - z]
h_list = [R(p.homogenize()) for p in f_list]

system_solve(h_list)

z^4 - 3*z^2*h^2 + 2*h^4


TypeError: polynomial must involve at most one variable

In [5]:
f1 = x^2 + x*y - 1
f2 = x*z + z^2
f3 = y^3 - y + z + 1 # these three polynomials define a system satisfying the hypotheses of Theorem 5
I = Ideal(f1,f2,f3)
G = I.groebner_basis()
G

[x^2 + x*y - 1, x*z + z^2, y^3 - y + z + 1, y*z + z^6 - 3*z^4 + z^3 + 3*z^2, z^7 - 3*z^5 + z^4 + 4*z^3 - z]

In [6]:
system_solve([f1,f2,f3])

[(1, 0, -1)]

In [7]:
R.<x1,x2,x3,x4,x5> = PolynomialRing(QQ, order = 'lex')
f1 = x1*x3^10 + x2^2*x4
f2 = x1^2 + x2 * x3^3 - 1
f3 = x1 + x4^2*x5^3 + 1
f4 = x2 - x3^2 + 2*x4^2 + 1
f5 = x3^2 + x5
f6 = x2*x3*x5

In [8]:
system_solve([f1,f2,f3,f4,f5,f6])

[(-1, -1, 0, 0, 0)]

In [58]:
R.<x,y,z> = PolynomialRing(QQ, order = 'degrevlex')
f1 = x^2 + x*y + z*y
f2 = x^3 + y^3 + z^3
I = Ideal(f1,f2)
system_solve([f1,f3])

[]

In [59]:
S.<t> = PolynomialRing(QQ)
from sage.rings.polynomial.hilbert import hilbert_poincare_series
hilbert_poincare_series(I)

(t + 1)/(t^2 - 2*t + 1)

In [60]:
((1 - t^2)*(1-t^3))/(1 - t)^3

(-t^3 - 2*t^2 - 2*t - 1)/(t - 1)

In [53]:
Ideal(f1).quotient(ideal(f2))

Ideal (x^2 + x*y + y*z) of Multivariate Polynomial Ring in x, y, z over Rational Field

In [51]:
I.dimension()

1

In [238]:
R.<x,y,z> = PolynomialRing(QQ, order = 'degrevlex')
f1 = x^5 + x*y
f2 = y^3
f3 = z^2
I = Ideal(f1,f2,f3)
hilbert_poincare_series(I)

t^7 + 3*t^6 + 5*t^5 + 6*t^4 + 6*t^3 + 5*t^2 + 3*t + 1

In [216]:
ideal(f1,f2).quotient(ideal(f3))

Ideal (y^3, x^5) of Multivariate Polynomial Ring in x, y, z over Rational Field

In [214]:
((1-t^5)*(1-t^3)*(1-t^2))/(1-t)^3

t^7 + 3*t^6 + 5*t^5 + 6*t^4 + 6*t^3 + 5*t^2 + 3*t + 1

In [107]:
I1 = ideal(f1)
I1.quotient(ideal(f2))

Ideal (x^2) of Multivariate Polynomial Ring in x, y, z over Rational Field

In [109]:
I2 = ideal(f1,f2)
I2.quotient(ideal(f3))

Ideal (y^2 + z^2, x^2) of Multivariate Polynomial Ring in x, y, z over Rational Field

In [225]:
R.<w,x,y,z> = PolynomialRing(GF(13), order = 'degrevlex')
f1 = x^2 - x
f2 = x*y - 1
f3 = w^6 - w
f4 = w^5*z^5 - 1
I = Ideal(f1,f2,f3,f4)
I.groebner_basis()

[w^5 - 1, z^5 - 1, x - 1, y - 1]

In [228]:
degree_reg(I,R)

11

In [237]:
len(I.homogenize()._singular_().mres(0).betti())/5

15

In [227]:
# computes degree of regularity for a homogeneous ideal
def hdegree_reg(I):
    G = I.groebner_basis()
    LT = Ideal([g.lt() for g in G])
    HS = hilbert_poincare_series(LT)
    delta = HS.numerator().degree()
    l = HS.denominator().degree()
    return(delta - l + 1)


# computes F^top
def f_top(I, R):
    f_list = I.gens()
    new_ring.<h> = PolynomialRing(R, order = 'degrevlex')
    homogenized = [new_ring(poly.homogenize()) for poly in f_list]
    final = [R(poly.subs(h=0)) for poly in homogenized] 
    return(final)


# computes degree of regularity in the nonhomogeneous case
def degree_reg(I, R):
    J = ideal(f_top(I,R))
    return hdegree_reg(J)

In [235]:
mac_grob([f1,f2,f3,f4],R)

0 [0] True False
1 [0] True False
2 [x^2 - x, x*y - 1] False False
3 [w*x^2 - w*x, x^3 - 1, w*x*y - w, x^2*y - 1, x*y^2 - y, x^2*z - x*z, x*y*z - z, x^2 - 1, x*y - 1, x - 1] False False
4 Polynomial Sequence with 29 Polynomials in 4 Variables True False
5 Polynomial Sequence with 65 Polynomials in 4 Variables True False
6 Polynomial Sequence with 126 Polynomials in 4 Variables True False
7 Polynomial Sequence with 222 Polynomials in 4 Variables True False
8 Polynomial Sequence with 363 Polynomials in 4 Variables True False
9 Polynomial Sequence with 559 Polynomials in 4 Variables True False
10 Polynomial Sequence with 822 Polynomials in 4 Variables False True
11 Polynomial Sequence with 1166 Polynomials in 4 Variables False True
12 Polynomial Sequence with 1603 Polynomials in 4 Variables False True
13 Polynomial Sequence with 2146 Polynomials in 4 Variables False True
14 Polynomial Sequence with 2810 Polynomials in 4 Variables False True
15 Polynomial Sequence with 3611 Polynomials in 

KeyboardInterrupt: 

In [336]:
def test_rowsp(f, f_list, d, ring):
    mons = get_mons(d, ring)
    M = mac_matrix(f_list, d, ring)
    done = False
    while not done:
        rowsp = M.row_space()
        to_be_added = []
        for row in M.rref().rows():
            g = 0
            for (coeff, mon) in zip(row, mons):
                g += coeff * mon
            max_test_deg = d - g.degree()
            if (g != 0) and (max_test_deg > 0):
                for mon in get_mons(max_test_deg, ring):
                    test_p = mon*g
                    test_vec = vector([test_p.monomial_coefficient(moon) for moon in mons])
                    if test_vec not in rowsp:
                        to_be_added.append(test_vec)
        if to_be_added == []:
            done = True
        else:
            new_rows = M.rows() + to_be_added
            M = Matrix(new_rows)
    f_vec = vector([ring(f).monomial_coefficient(mon) for mon in mons])
    return f_vec in M.row_space()

def get_mons(d, ring, leq = True):
    if leq:
        mons = []
        for i in range(d+1):
            mons += [ring({tuple(a):1}) for a in WeightedIntegerVectors(i,[1 for gen in ring.gens()])]
    else:
        mons = [ring({tuple(a):1}) for a in WeightedIntegerVectors(d,[1 for gen in ring.gens()])]
    mons.sort()
    mons.reverse()
    return mons

def deg_fall(f, f_list, ring):
    test_deg = f.degree()
    while True:
        print(test_deg, time.strftime("%H:%M:%S", time.localtime()))
        if test_rowsp(f, f_list, test_deg, ring):
            return test_deg
        test_deg += 1
        
def lfd(f_list, ring):
    fall_degrees = []
    I = ideal(f_list)
    B = I.groebner_basis()
    print(B)
    for f in B:
        print('\n')
        print(f)
        d = deg_fall(f, f_list, ring)
        if d > f.degree():
            fall_degrees.append(d)
    if fall_degrees == []:
        return 0
    else:
        return max(fall_degrees)

In [223]:
deg_fall(y - 1, [f1,f2,f3,f4], R)

[w, x, y, z, 1]
[w^2, w*x, x^2, w*y, x*y, y^2, w*z, x*z, y*z, z^2, w, x, y, z, 1]
[w^3, w^2*x, w*x^2, x^3, w^2*y, w*x*y, x^2*y, w*y^2, x*y^2, y^3, w^2*z, w*x*z, x^2*z, w*y*z, x*y*z, y^2*z, w*z^2, x*z^2, y*z^2, z^3, w^2, w*x, x^2, w*y, x*y, y^2, w*z, x*z, y*z, z^2, w, x, y, z, 1]


3

In [581]:
# Compute the Macaulay matrix in degree d of a system f = [f1,...,fs]
def mac_matrix(f,d,ring):
    mons0 = monomials(list(ring.gens()), [d+1 for i in range(len(ring.gens()))])
    mons = []
    for mon in mons0:
        if mon.degree() <= d:
            mons.append(mon)
    mons.sort()
    mons.reverse() # for reasons I don't understand, the .sort() method respects the chosen order but reverses it
    col_labs = []
    for deg in range(d + 1):
        segment = []
        for poly in f:
            for mon in mons:
                label = ring(mon)*ring(poly)
                if label.degree() == deg:
                    segment.append(label)
        col_labs += segment
    return matrix([[label.monomial_coefficient(mon) for mon in mons] for label in col_labs])


# Takes an RREF Macaulay matrix as input and returns a list of the corresponding polynomials
def recover_polys(M, d, ring):
    mons0 = monomials(list(ring.gens()), [d+1 for i in range(len(ring.gens()))])
    mons = []
    for mon in mons0:
        if mon.degree() <= d:
            mons.append(mon)
    mons.sort()
    mons.reverse()
    B = []
    for row in M.rows():
        g = 0
        for (coeff,mon) in zip(row,mons):
            g += coeff*mon
        if g != 0:
            B.append(g)
    if B == []:
        B = [ring(0)]
    return Sequence(B) # Sage doesn't know how to work with the output list unless it is a "Sequence" of polynomials


# Now instead of using mac_matrix followed by recover_polys, we ask for the new polynomials directly:
def mac_basis(f,d,ring):
    mons0 = monomials(list(ring.gens()), [d+1 for i in range(len(ring.gens()))])
    mons = []
    for mon in mons0:
        if mon.degree() <= d:
            mons.append(mon)
    mons.sort()
    mons.reverse() # for reasons I don't understand, the .sort() method respects the chosen order but reverses it
    col_labs = []
    for deg in range(d + 1):
        segment = []
        for poly in f:
            for mon in mons:
                label = ring(mon)*ring(poly)
                if label.degree() == deg:
                    segment.append(label)
        col_labs += segment
    M = matrix([[label.monomial_coefficient(mon) for mon in mons] for label in col_labs]).rref()
    B = []
    for row in M.rows():
        g = 0
        for (coeff,mon) in zip(row,mons):
            g += coeff*mon
        if g != 0:
            B.append(g)
    if B == []:
        B = [ring(0)]
    return Sequence(B)


# Now define a function that calculates a Gröbner basis and the solving degree (per Caminata and Gorla, 2021)
def mac_grob(f,ring):
    test_deg = 0
    while True:
        B = mac_basis(f, test_deg, ring)
        print(test_deg, B, B.is_groebner(), ideal(B) == ideal(f))
        if B.is_groebner() and ideal(B) == ideal(f):
            return (test_deg,B)
        test_deg += 1
        

# Modified version to compute the solving degree per Caminata and Gorla, 2023
def sdeg(f, ring):
    d = 0
    while True:
        d_level_base = []
        mons = get_mons(d, ring)
        M = mac_matrix(f, d, ring)
        done = False
        while not done:
            rowsp = M.row_space()
            to_be_added = []
            for row in M.rref().rows():
                g = 0
                for (coeff, mon) in zip(row, mons):
                    g += coeff * mon
                max_test_deg = d - g.degree()
                if (g != 0) and (max_test_deg > 0):
                    for mon in get_mons(max_test_deg, ring):
                        test_p = mon*g
                        test_vec = vector([test_p.monomial_coefficient(moon) for moon in mons])
                        if test_vec not in rowsp:
                            to_be_added.append(test_vec)
            if to_be_added == []:
                done = True
                for row in M.rref().rows():
                    g = 0
                    for (coeff, mon) in zip(row, mons):
                        g += coeff * mon
                    if g != 0:
                        d_level_base.append(g)
                if d_level_base == []:
                    d_level_base = [ring(0)]
                check_1 = Sequence(d_level_base).is_groebner()
                check_2 = (ideal(d_level_base) == ideal(f))
                print(d, Sequence(d_level_base), check_1, check_2)
                if check_1 and check_2:
                    return (d, Sequence(d_level_base))
                d += 1
            else:
                new_rows = M.rows() + to_be_added
                M = Matrix(new_rows)

In [263]:
R.<x,y,z> = PolynomialRing(QQ, order = 'degrevlex')
f1 = x^5 + x*y
f2 = y^3*x
f3 = z^2 + x^4
I = Ideal(f1,f2,f3)
I.groebner_basis()

[y^3*z^2, x^4 + z^2, x*y^3, z^4 - y*z^2, x*z^2 - x*y]

In [252]:
deg_fall(x*z^2 - x*y, [f1,f2,f3], R)

[x^3, x^2*y, x*y^2, y^3, x^2*z, x*y*z, y^2*z, x*z^2, y*z^2, z^3, x^2, x*y, y^2, x*z, y*z, z^2, x, y, z, 1]
[x^4, x^3*y, x^2*y^2, x*y^3, y^4, x^3*z, x^2*y*z, x*y^2*z, y^3*z, x^2*z^2, x*y*z^2, y^2*z^2, x*z^3, y*z^3, z^4, x^3, x^2*y, x*y^2, y^3, x^2*z, x*y*z, y^2*z, x*z^2, y*z^2, z^3, x^2, x*y, y^2, x*z, y*z, z^2, x, y, z, 1]
[x^5, x^4*y, x^3*y^2, x^2*y^3, x*y^4, y^5, x^4*z, x^3*y*z, x^2*y^2*z, x*y^3*z, y^4*z, x^3*z^2, x^2*y*z^2, x*y^2*z^2, y^3*z^2, x^2*z^3, x*y*z^3, y^2*z^3, x*z^4, y*z^4, z^5, x^4, x^3*y, x^2*y^2, x*y^3, y^4, x^3*z, x^2*y*z, x*y^2*z, y^3*z, x^2*z^2, x*y*z^2, y^2*z^2, x*z^3, y*z^3, z^4, x^3, x^2*y, x*y^2, y^3, x^2*z, x*y*z, y^2*z, x*z^2, y*z^2, z^3, x^2, x*y, y^2, x*z, y*z, z^2, x, y, z, 1]


5

In [264]:
lfd([f1,f2,f3],R)

[x^5, x^4*y, x^3*y^2, x^2*y^3, x*y^4, y^5, x^4*z, x^3*y*z, x^2*y^2*z, x*y^3*z, y^4*z, x^3*z^2, x^2*y*z^2, x*y^2*z^2, y^3*z^2, x^2*z^3, x*y*z^3, y^2*z^3, x*z^4, y*z^4, z^5, x^4, x^3*y, x^2*y^2, x*y^3, y^4, x^3*z, x^2*y*z, x*y^2*z, y^3*z, x^2*z^2, x*y*z^2, y^2*z^2, x*z^3, y*z^3, z^4, x^3, x^2*y, x*y^2, y^3, x^2*z, x*y*z, y^2*z, x*z^2, y*z^2, z^3, x^2, x*y, y^2, x*z, y*z, z^2, x, y, z, 1]
[x^6, x^5*y, x^4*y^2, x^3*y^3, x^2*y^4, x*y^5, y^6, x^5*z, x^4*y*z, x^3*y^2*z, x^2*y^3*z, x*y^4*z, y^5*z, x^4*z^2, x^3*y*z^2, x^2*y^2*z^2, x*y^3*z^2, y^4*z^2, x^3*z^3, x^2*y*z^3, x*y^2*z^3, y^3*z^3, x^2*z^4, x*y*z^4, y^2*z^4, x*z^5, y*z^5, z^6, x^5, x^4*y, x^3*y^2, x^2*y^3, x*y^4, y^5, x^4*z, x^3*y*z, x^2*y^2*z, x*y^3*z, y^4*z, x^3*z^2, x^2*y*z^2, x*y^2*z^2, y^3*z^2, x^2*z^3, x*y*z^3, y^2*z^3, x*z^4, y*z^4, z^5, x^4, x^3*y, x^2*y^2, x*y^3, y^4, x^3*z, x^2*y*z, x*y^2*z, y^3*z, x^2*z^2, x*y*z^2, y^2*z^2, x*z^3, y*z^3, z^4, x^3, x^2*y, x*y^2, y^3, x^2*z, x*y*z, y^2*z, x*z^2, y*z^2, z^3, x^2, x*y, y^2, x*z, 

7

In [265]:
mac_grob([f1,f2,f3],R)

0 [0] True False
1 [0] True False
2 [0] True False
3 [0] True False
4 [x^4 + z^2, x*y^3] False False
5 [x^5 + x*y, x^4*y + y*z^2, x^2*y^3, x*y^4, x^4*z + z^3, x*y^3*z, x^4 + z^2, x*y^3, x*z^2 - x*y] False True
6 Polynomial Sequence with 24 Polynomials in 3 Variables False True
7 Polynomial Sequence with 50 Polynomials in 3 Variables False True
8 Polynomial Sequence with 88 Polynomials in 3 Variables True True


(8, Polynomial Sequence with 88 Polynomials in 3 Variables)

In [275]:
sdeg([f1,f2,f3],R)

0 [0] True False
1 [0] True False
2 [0] True False
3 [0] True False
4 [x^4 + z^2, x*y^3] False False
5 [x^5 + x*y, x^4*y + y*z^2, x^2*y^3, x*y^4, x^4*z + z^3, x*y^3*z, x^3*z^2 - x^3*y, x^2*y*z^2 - x^2*y^2, x*y^2*z^2, x^2*z^3 - x^2*y*z, x*y*z^3 - x*y^2*z, x*z^4 - x*y^2, x^4 + z^2, x*y^3, x^2*z^2 - x^2*y, x*y*z^2 - x*y^2, x*z^3 - x*y*z, x*z^2 - x*y] False True
6 Polynomial Sequence with 44 Polynomials in 3 Variables False True
7 Polynomial Sequence with 81 Polynomials in 3 Variables True True


(7, Polynomial Sequence with 81 Polynomials in 3 Variables)

In [259]:
mac_matrix([f1,f2,f3],4,R).rref()

[x^4, x^3*y, x^2*y^2, x*y^3, y^4, x^3*z, x^2*y*z, x*y^2*z, y^3*z, x^2*z^2, x*y*z^2, y^2*z^2, x*z^3, y*z^3, z^4, x^3, x^2*y, x*y^2, y^3, x^2*z, x*y*z, y^2*z, x*z^2, y*z^2, z^3, x^2, x*y, y^2, x*z, y*z, z^2, x, y, z, 1]


[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

In [261]:
(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0) in span(mac_matrix([f1,f2,f3],4,R).rows())

[x^4, x^3*y, x^2*y^2, x*y^3, y^4, x^3*z, x^2*y*z, x*y^2*z, y^3*z, x^2*z^2, x*y*z^2, y^2*z^2, x*z^3, y*z^3, z^4, x^3, x^2*y, x*y^2, y^3, x^2*z, x*y*z, y^2*z, x*z^2, y*z^2, z^3, x^2, x*y, y^2, x*z, y*z, z^2, x, y, z, 1]


False

In [288]:
# Example 4.1
R.<x,y> = PolynomialRing(GF(103), order = "degrevlex")
f1 = x*y + y
f2 = y^2 - 1
f3 = x^102 - 1
I = Ideal(f1,f2,f3)
print(I.groebner_basis(), '\n')
print(degree_reg(I,R), '\n')
print(mac_grob([f1,f2,f3],R), '\n')
print(sdeg([f1,f2,f3],R), '\n')
print(lfd([f1,f2,f3],R), '\n')

[y^2 - 1, x + 1] 

102 

0 [0] True False
1 [0] True False
2 [x*y + y, y^2 - 1] False True
3 [x^2*y - y, x*y^2 + 1, y^3 - y, x*y + y, y^2 - 1, x + 1] True True
(3, [x^2*y - y, x*y^2 + 1, y^3 - y, x*y + y, y^2 - 1, x + 1]) 

0 [0] True False
1 [0] True False
2 [x*y + y, y^2 - 1] False True
3 [x^3 + 1, x^2*y - y, x*y^2 + 1, y^3 - y, x^2 - 1, x*y + y, y^2 - 1, x + 1] True True
(3, [x^3 + 1, x^2*y - y, x*y^2 + 1, y^3 - y, x^2 - 1, x*y + y, y^2 - 1, x + 1]) 

3 



In [291]:
# Example 4.2
R.<x,y,z> = PolynomialRing(GF(11), order = "degrevlex")
f1 = x*y + y
f2 = y^2 - 1
f3 = z^10 - 1
I = Ideal(f1,f2,f3)
print(I.groebner_basis(), '\n')
print(degree_reg(I,R), '\n')
print(mac_grob([f1,f2,f3],R), '\n')
print(sdeg([f1,f2,f3],R), '\n')
print(lfd([f1,f2,f3],R), '\n')

[z^10 - 1, y^2 - 1, x + 1] 

11 

0 [0] True False
1 [0] True False
2 [x*y + y, y^2 - 1] False False
3 [x^2*y - y, x*y^2 + 1, y^3 - y, x*y*z + y*z, y^2*z - z, x*y + y, y^2 - 1, x + 1] True False
4 [x^3*y + y, x^2*y^2 - 1, x*y^3 + y, y^4 - 1, x^2*y*z - y*z, x*y^2*z + z, y^3*z - y*z, x*y*z^2 + y*z^2, y^2*z^2 - z^2, x^2*y - y, x*y^2 + 1, y^3 - y, x*y*z + y*z, y^2*z - z, x^2 - 1, x*y + y, y^2 - 1, x*z + z, x + 1] True False
5 Polynomial Sequence with 36 Polynomials in 3 Variables True False
6 Polynomial Sequence with 60 Polynomials in 3 Variables True False
7 Polynomial Sequence with 92 Polynomials in 3 Variables True False
8 Polynomial Sequence with 133 Polynomials in 3 Variables True False
9 Polynomial Sequence with 184 Polynomials in 3 Variables True False
10 Polynomial Sequence with 247 Polynomials in 3 Variables True True
(10, Polynomial Sequence with 247 Polynomials in 3 Variables) 

0 [0] True False
1 [0] True False
2 [x*y + y, y^2 - 1] False False
3 [x^3 + 1, x^2*y - y, x*y^2 + 1, 

In [295]:
# Example 4.3
R.<x,y> = PolynomialRing(GF(11), order = "degrevlex")
f1 = x^2 - 1
f2 = y^2 - 1
I = Ideal(f1,f2)
print(I.groebner_basis(), '\n')
print(degree_reg(I,R), '\n')
print(mac_grob([f1,f2],R), '\n')
print(sdeg([f1,f2],R), '\n')
print(lfd([f1,f2],R), '\n')
I.homogenize()._singular_().mres(0).betti()

[x^2 - 1, y^2 - 1] 

3 

0 [0] True False
1 [0] True False
2 [x^2 - 1, y^2 - 1] True True
(2, [x^2 - 1, y^2 - 1]) 

0 [0] True False
1 [0] True False
2 [x^2 - 1, y^2 - 1] True True
(2, [x^2 - 1, y^2 - 1]) 

0 



     1     0     0
     0     2     0
     0     0     1

In [296]:
# Example 4.4
R.<w,x,y,z> = PolynomialRing(GF(5), order = "degrevlex")
f1 = x^2 - 1
f2 = x*y - 1
f3 = w^5 - w
f4 = w^4*z^4 - 1
I = Ideal(f1,f2,f3,f4)
print(I.groebner_basis(), '\n')
print(degree_reg(I,R), '\n')
print(mac_grob([f1,f2,f3,f4],R), '\n')
print(sdeg([f1,f2,f3,f4],R), '\n')
print(lfd([f1,f2,f3,f4],R), '\n')
I.homogenize()._singular_().mres(0).betti()

[w^4 - 1, z^4 - 1, y^2 - 1, x - y] 

9 

0 [0] True False
1 [0] True False
2 [x^2 - 1, x*y - 1] False False
3 [w*x^2 - w, x^3 - y, w*x*y - w, x^2*y - y, x*y^2 - y, x^2*z - z, x*y*z - z, x^2 - 1, x*y - 1, x - y] False False
4 Polynomial Sequence with 29 Polynomials in 4 Variables True False
5 Polynomial Sequence with 66 Polynomials in 4 Variables True False
6 Polynomial Sequence with 130 Polynomials in 4 Variables True False
7 Polynomial Sequence with 230 Polynomials in 4 Variables True False
8 Polynomial Sequence with 376 Polynomials in 4 Variables False True
9 Polynomial Sequence with 580 Polynomials in 4 Variables False True
10 Polynomial Sequence with 853 Polynomials in 4 Variables False True
11 Polynomial Sequence with 1206 Polynomials in 4 Variables False True
12 Polynomial Sequence with 1652 Polynomials in 4 Variables False True
13 Polynomial Sequence with 2204 Polynomials in 4 Variables False True
14 Polynomial Sequence with 2876 Polynomials in 4 Variables False True
15 Polynomi

     1     0     0     0     0
     0     2     0     0     0
     0     0     1     0     0
     0     0     0     0     0
     0     1     0     0     0
     0     0     2     0     0
     0     0     0     1     0
     0     1     0     0     0
     0     0     3     1     0
     0     0     0     2     1
     0     0     0     0     0
     0     0     1     1     0
     0     0     0     1     1

In [298]:
# Example from Minko's Thesis
R.<x,y,z> = PolynomialRing(GF(7), order = 'degrevlex')
f1 = x^5 + y^5 + z^5 - 1
f2 = x^3 + y^3 + z^2 - 1
f3 = y^6 - 1
f4 = z^6 - 1
f_list = [f1*f1*f1, f1*f1*f2, f1*f1*f3, f1*f1*f4, f1*f2*f2, f1*f2*f3, f1*f2*f4, f1*f3*f3, f1*f3*f4, f1*f4*f4, f2*f2*f2, f2*f2*f3, f2*f2*f4, f2*f3*f3, f2*f3*f4, f2*f4*f4, f3*f3*f3, f3*f3*f4, f3*f4*f4, f4*f4*f4, x^7 - x, y^7 - y, z^7 - z]
I = Ideal(f_list)
print(I.groebner_basis(), '\n')
print(degree_reg(I,R), '\n')
print(mac_grob(f_list,R), '\n')
print(sdeg(f_list,R), '\n')
print(lfd(f_list,R), '\n')
I.homogenize()._singular_().mres(0).betti()

[y^6 - 1, y^2*z - y^2 - x*z + y*z - 3*x + 2*y + z - 1, x^2 - y^2 + 2*y*z - 2*x + 3*y - 3*z + 3, x*y + y^2 - x*z - 2*y*z + 3*x - 3*y + 3*z - 3, z^2 + 2*x + 2*y - 2*z + 1] 

15 

0 [0] True False
1 [0] True False
2 [0] True False
3 [0] True False
4 [0] True False
5 [0] True False
6 [0] True False
7 [x^7 - x, y^7 - y, z^7 - z] True False
8 [x^8 - x^2, x^7*y - x*y, x*y^7 - x*y, y^8 - y^2, x^7*z - x*z, y^7*z - y*z, x*z^7 - x*z, y*z^7 - y*z, z^8 - z^2, x^7 - x, y^7 - y, z^7 - z] True False
9 Polynomial Sequence with 31 Polynomials in 3 Variables False False
10 Polynomial Sequence with 64 Polynomials in 3 Variables False False
11 Polynomial Sequence with 116 Polynomials in 3 Variables False False
12 Polynomial Sequence with 194 Polynomials in 3 Variables False False
13 Polynomial Sequence with 304 Polynomials in 3 Variables False False


IOStream.flush timed out


14 Polynomial Sequence with 450 Polynomials in 3 Variables False False
15 Polynomial Sequence with 636 Polynomials in 3 Variables False False
16 Polynomial Sequence with 862 Polynomials in 3 Variables False False
17 Polynomial Sequence with 1102 Polynomials in 3 Variables False False
18 Polynomial Sequence with 1304 Polynomials in 3 Variables False True
19 Polynomial Sequence with 1521 Polynomials in 3 Variables False True
20 Polynomial Sequence with 1757 Polynomials in 3 Variables False True
21 Polynomial Sequence with 2013 Polynomials in 3 Variables False True
22 Polynomial Sequence with 2290 Polynomials in 3 Variables True True
(22, Polynomial Sequence with 2290 Polynomials in 3 Variables) 

0 [0] True False
1 [0] True False
2 [0] True False
3 [0] True False
4 [0] True False
5 [0] True False
6 [0] True False
7 [x^7 - x, y^7 - y, z^7 - z] True False
8 [x^8 - x^2, x^7*y - x*y, x*y^7 - x*y, y^8 - y^2, x^7*z - x*z, y^7*z - y*z, x*z^7 - x*z, y*z^7 - y*z, z^8 - z^2, x^7 - x, y^7 - y, z^7 

     1     0     0     0     0
     0     0     0     0     0
     0     0     0     0     0
     0     0     0     0     0
     0     0     0     0     0
     0     0     0     0     0
     0     3     0     0     0
     0     0     0     0     0
     0     1     0     0     0
     0     0     0     0     0
     0     1     0     0     0
     0     2     2     0     0
     0     1     4     0     0
     0     2     4     0     0
     0     4     6     1     0
     0     2    29     4     0
     0     2    33   116    61
     0     3    11    15     6
     0     0     3     5     2
     0     0     2     2     0
     0     0     0     1     0
     0     0     0     0     1

In [572]:
R.<x,y,z> = PolynomialRing(QQ, order = 'degrevlex')
f1 = y^2
f2 = y*z + x
I = Ideal(f1,f2)
I.groebner_basis()

[x^2, x*y, y^2, y*z + x]

In [543]:
mac_grob([f1,f2],R)

0 [0] True False
1 [0] True False
2 [y^2, y*z + x] False True
3 [x*y^2, y^3, x*y*z + x^2, y^2*z, y*z^2 + x*z, x*y, y^2, y*z + x] False True
4 [x^2*y^2, x*y^3, y^4, x^2*y*z + x^3, x*y^2*z, y^3*z, x*y*z^2 + x^2*z, y^2*z^2, y*z^3 + x*z^2, x^2*y, x*y^2, y^3, x*y*z, y^2*z, y*z^2 + x*z, x^2, x*y, y^2, y*z + x] True True


(4,
 [x^2*y^2, x*y^3, y^4, x^2*y*z + x^3, x*y^2*z, y^3*z, x*y*z^2 + x^2*z, y^2*z^2, y*z^3 + x*z^2, x^2*y, x*y^2, y^3, x*y*z, y^2*z, y*z^2 + x*z, x^2, x*y, y^2, y*z + x])

In [567]:
S.<x,y,z,h> = PolynomialRing(QQ, order = "degrevlex")
f1 = y*z + x*h
f2 = y^2
K = ideal(f1,f2)
M = ideal(x,y,z,h)
J = K.saturation(M)
h in J

False

In [544]:
I.homogenize()._singular_().mres(0).betti()

     1     0     0
     0     2     0
     0     0     1

In [545]:
degree_reg(I,R)

1

In [577]:
sdeg([f1,f2],R)

[]

[]

0 [0] True False
[]

[]

1 [0] True False
[0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 1 0 1 0 0 0]

[0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 1 0 1 0 0 0]

2 [y^2, y*z + x] False True
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0]

[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0]

[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0

(3,
 [x^3, x^2*y, x*y^2, y^3, x^2*z, x*y*z, y^2*z, y*z^2 + x*z, x^2, x*y, y^2, y*z + x])

In [548]:
lfd([f1,f2],R)

[x^2, x*y, y^2, y*z + x]


x^2
2 19:21:27
3 19:21:27


x*y
2 19:21:27
3 19:21:27


y^2
2 19:21:27


y*z + x
2 19:21:27


3

In [554]:
mac_matrix([f1,f2],3,R)

[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0]

In [553]:
vector([0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]) in M.row_space()

False

In [555]:
mac_matrix([f1,f2],3,R).rref()

[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0]

In [556]:
get_mons(3,R)

[x^3,
 x^2*y,
 x*y^2,
 y^3,
 x^2*z,
 x*y*z,
 y^2*z,
 x*z^2,
 y*z^2,
 z^3,
 x^2,
 x*y,
 y^2,
 x*z,
 y*z,
 z^2,
 x,
 y,
 z,
 1]

In [562]:
print(mac_matrix([f1,f2],4,R).str())

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [560]:
get_mons(4,R)

[x^4,
 x^3*y,
 x^2*y^2,
 x*y^3,
 y^4,
 x^3*z,
 x^2*y*z,
 x*y^2*z,
 y^3*z,
 x^2*z^2,
 x*y*z^2,
 y^2*z^2,
 x*z^3,
 y*z^3,
 z^4,
 x^3,
 x^2*y,
 x*y^2,
 y^3,
 x^2*z,
 x*y*z,
 y^2*z,
 x*z^2,
 y*z^2,
 z^3,
 x^2,
 x*y,
 y^2,
 x*z,
 y*z,
 z^2,
 x,
 y,
 z,
 1]

In [585]:
# This is Example 5 in Caminata and Gorla, with the field equations

S.<x1,x2,x3> = PolynomialRing(GF(5), order = "degrevlex")
f1 = x3^2 - x2
f2 = x2^3 - x1
f3 = x1^5 - x1
f4 = x2^5 - x2
f5 = x3^5 - x3
I = ideal(f1,f2,f3,f4,f5)
I.groebner_basis()

[x2^3 - x2, x2^2*x3 - x3, x3^2 - x2, x1 - x2]

In [586]:
mac_grob([f1,f2,f3,f4,f5],S)

0 [0] True False
1 [0] True False
2 [x3^2 - x2] True False
3 [x2^3 - x1, x1*x3^2 - x1*x2, x2*x3^2 - x2^2, x3^3 - x2*x3, x3^2 - x2] True False
4 [x1*x2^3 - x1^2, x2^4 - x1*x2, x2^3*x3 - x1*x3, x1^2*x3^2 - x1^2*x2, x1*x2*x3^2 - x1*x2^2, x2^2*x3^2 - x1, x1*x3^3 - x1*x2*x3, x2*x3^3 - x2^2*x3, x3^4 - x2^2, x2^3 - x1, x1*x3^2 - x1*x2, x2*x3^2 - x2^2, x3^3 - x2*x3, x3^2 - x2] True False
5 Polynomial Sequence with 32 Polynomials in 3 Variables False True
6 Polynomial Sequence with 62 Polynomials in 3 Variables True True


(6, Polynomial Sequence with 62 Polynomials in 3 Variables)

In [587]:
sdeg([f1,f2,f3,f4,f5],S)

0 [0] True False
1 [0] True False
2 [x3^2 - x2] True False
3 [x2^3 - x1, x1*x3^2 - x1*x2, x2*x3^2 - x2^2, x3^3 - x2*x3, x3^2 - x2] True False
4 [x1*x2^3 - x1^2, x2^4 - x1*x2, x2^3*x3 - x1*x3, x1^2*x3^2 - x1^2*x2, x1*x2*x3^2 - x1*x2^2, x2^2*x3^2 - x1, x1*x3^3 - x1*x2*x3, x2*x3^3 - x2^2*x3, x3^4 - x2^2, x2^3 - x1, x1*x3^2 - x1*x2, x2*x3^2 - x2^2, x3^3 - x2*x3, x3^2 - x2] True False
5 Polynomial Sequence with 51 Polynomials in 3 Variables True True


(5, Polynomial Sequence with 51 Polynomials in 3 Variables)

In [588]:
J = ideal(f1,f2)
J.groebner_basis()

[x2^3 - x1, x3^2 - x2]

In [592]:
mac_grob([f1,f2],S)

0 [0] True False
1 [0] True False
2 [x3^2 - x2] True False
3 [x2^3 - x1, x1*x3^2 - x1*x2, x2*x3^2 - x2^2, x3^3 - x2*x3, x3^2 - x2] True True


(3, [x2^3 - x1, x1*x3^2 - x1*x2, x2*x3^2 - x2^2, x3^3 - x2*x3, x3^2 - x2])

In [593]:
sdeg([f1,f2],S)

0 [0] True False
1 [0] True False
2 [x3^2 - x2] True False
3 [x2^3 - x1, x1*x3^2 - x1*x2, x2*x3^2 - x2^2, x3^3 - x2*x3, x3^2 - x2] True True


(3, [x2^3 - x1, x1*x3^2 - x1*x2, x2*x3^2 - x2^2, x3^3 - x2*x3, x3^2 - x2])

In [615]:
# Example from Minko's Thesis
R.<x,y,z> = PolynomialRing(GF(7), order = 'degrevlex')
f1 = x^5 + y^5 + z^5 - 1
f2 = x^3 + y^3 + z^2 - 1
f3 = y^6 - 1
f4 = z^6 - 1
f_list = [f1*f1*f1, f1*f1*f2, f1*f1*f3, f1*f1*f4, f1*f2*f2, f1*f2*f3, f1*f2*f4, f1*f3*f3, f1*f3*f4, f1*f4*f4, f2*f2*f2, f2*f2*f3, f2*f2*f4, f2*f3*f3, f2*f3*f4, f2*f4*f4, f3*f3*f3, f3*f3*f4, f3*f4*f4, f4*f4*f4, x^7 - x, y^7 - y, z^7 - z]
h_list = [p.homogenize() for p in f_list]
Fh = ideal(h_list)
in_grob = [p.lt() for p in Fh.groebner_basis()]
intl_Fh = ideal(in_grob)

In [619]:
len(ideal(in_grob)._singular_().mres(0).betti())/5

22

IOStream.flush timed out


Polynomial Sequence with 213 Polynomials in 4 Variables

In [624]:
S.<w,x,y,z,h> = PolynomialRing(QQ, order = "degrevlex")
f1 = x^2 - x
f2 = x*y - 1
f3 = w^6 - w
f4 = w^5*z^5 - 1
h_list = [f.homogenize() for f in [f1,f2,f3,f4]]
Fh = ideal(h_list)
in_list = [f.lt() for f in Fh.groebner_basis()]
len(ideal(in_list)._singular_().mres(0).betti())/6

20

In [626]:
len(ideal(h_list)._singular_().mres(0).betti())/5

15

In [659]:
# Example from Minko's Thesis
R.<x,y,z> = PolynomialRing(GF(7), order = 'lex')
f1 = x^5 + y^5 + z^5 - 1
f2 = x^3 + y^3 + z^2 - 1
f3 = y^6 - 1
f4 = z^6 - 1
f_list = [f1*f1*f1, f1*f1*f2, f1*f1*f3, f1*f1*f4, f1*f2*f2, f1*f2*f3, f1*f2*f4, f1*f3*f3, f1*f3*f4, f1*f4*f4, f2*f2*f2, f2*f2*f3, f2*f2*f4, f2*f3*f3, f2*f3*f4, f2*f4*f4, f3*f3*f3, f3*f3*f4, f3*f4*f4, f4*f4*f4, x^7 - x, y^7 - y, z^7 - z]
h_list = [p.homogenize() for p in f_list]
Fh = ideal(h_list)
ideal(f_list).dimension()

0

In [664]:
for p in ideal(h_list).groebner_basis():
    print(p)

x^7 - x*h^6
x^6*y^3 + x^6*z^2*h - x^6*h^3 + x^3*y^6 + 2*x^3*y^3*z^2*h - 2*x^3*y^3*h^3 + x^3*z^4*h^2 - 2*x^3*z^2*h^4 - x^3*h^6 + y^6*z^2*h - y^6*h^3 + y^3*z^4*h^2 - 2*y^3*z^2*h^4 - y^3*h^6 - 2*z^6*h^3 - z^4*h^5 + z^2*h^7 + 2*h^9
x^6*y^2*z^2*h - x^6*y^2*h^3 - x^6*z^5 + x^6*h^5 - 3*x^5*h^6 - 2*x^3*y^3*z^5 + 2*x^3*y^3*h^5 + x^3*y^2*z^4*h^2 - 2*x^3*y^2*z^2*h^4 - 2*x^3*y^2*h^6 + 2*x^3*z^5*h^3 + 2*x^3*z^2*h^6 - 2*x^3*z*h^7 - 2*x^3*h^8 + x^2*y^6*z^2*h - x^2*y^6*h^3 + x^2*y^3*z^4*h^2 - 2*x^2*y^3*z^2*h^4 - 2*x^2*y^3*h^6 - 2*x^2*z^6*h^3 - x^2*z^4*h^5 + 3*x^2*h^9 - y^6*z^5 + y^6*h^5 - 3*y^5*h^6 + 2*y^3*z^5*h^3 + 2*y^3*z^2*h^6 - 2*y^3*z*h^7 - 2*y^3*h^8 - 2*y^2*z^6*h^3 - y^2*z^4*h^5 + 3*y^2*h^9 - z^5*h^6 + z^4*h^7 - z^3*h^8 - 2*z^2*h^9 + 2*z*h^10 + h^11
x^6*y*z^5 - x^6*y*h^5 + 3*x^5*y*h^6 - x^3*y^6*z^2*h + x^3*y^6*h^3 + 2*x^3*y^4*z^5 - 2*x^3*y^4*h^5 - x^3*y^3*z^4*h^2 + 2*x^3*y^3*z^2*h^4 + x^3*y^3*h^6 - 2*x^3*y*z^5*h^3 - 2*x^3*y*z^2*h^6 + 2*x^3*y*z*h^7 + 2*x^3*y*h^8 + 2*x^3*z^6*h^3 + x^3*z^4*h^5 - x^

In [665]:
system_solve(h_list)

z^4 - 3*z^2*h^2 + 2*h^4


TypeError: polynomial must involve at most one variable